In [1]:
import numpy as np
import gymnasium as gym
from collections import defaultdict  # required for creating Q(s, a)
from moviepy import ImageSequenceClip # to generate gif
from IPython.display import Image
import math
import matplotlib 
#matplotlib.use('Qt5Agg') # Activte it if you want external plot for any interaction
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

### 🏋️‍♂️ CartPole-v1 Discretization Function

CartPole has **4 continuous state variables**:

 **1**. Cart position

 **2**. Cart velocity

 **3**. Pole angle

 **4**. Pole angular velocity

We’ll discretize these into bins.

In [2]:


def discretize_cartpole_state(observation, state_bins):
    """
    Convert a continuous CartPole state into a discrete index for Q-learning.

    Parameters:
        observation : array-like of shape (4,)
            [cart_position, cart_velocity, pole_angle, pole_angular_velocity]
        state_bins : list or array-like of length 4
            Number of bins for each state dimension, e.g. [6, 6, 12, 6]

    Returns:
        int
            A single integer index representing the discretized state.
    """
    cart_pos, cart_vel, pole_ang, pole_ang_vel = observation

    # Define reasonable thresholds for state variables
    cart_pos_limit = 4.8       # track ends at about ±4.8
    cart_vel_limit = 3.0       # max reasonable velocity
    pole_ang_limit = math.radians(15)  # about ±15 degrees
    pole_ang_vel_limit = math.radians(50)

    # Create bin edges for each state variable
    cart_pos_bins = np.linspace(-cart_pos_limit, cart_pos_limit, state_bins[0] + 1)
    cart_vel_bins = np.linspace(-cart_vel_limit, cart_vel_limit, state_bins[1] + 1)
    pole_ang_bins = np.linspace(-pole_ang_limit, pole_ang_limit, state_bins[2] + 1)
    pole_ang_vel_bins = np.linspace(-pole_ang_vel_limit, pole_ang_vel_limit, state_bins[3] + 1)

    # Digitize each continuous value into a discrete bucket
    cart_pos_idx = np.digitize(cart_pos, cart_pos_bins) - 1
    cart_vel_idx = np.digitize(cart_vel, cart_vel_bins) - 1
    pole_ang_idx = np.digitize(pole_ang, pole_ang_bins) - 1
    pole_ang_vel_idx = np.digitize(pole_ang_vel, pole_ang_vel_bins) - 1

    # Clip indices to avoid overflow (in case value exceeds limits)
    cart_pos_idx = np.clip(cart_pos_idx, 0, state_bins[0] - 1)
    cart_vel_idx = np.clip(cart_vel_idx, 0, state_bins[1] - 1)
    pole_ang_idx = np.clip(pole_ang_idx, 0, state_bins[2] - 1)
    pole_ang_vel_idx = np.clip(pole_ang_vel_idx, 0, state_bins[3] - 1)

    # Combine all indices into a single integer (multi-dimensional → 1D)
    index = (cart_pos_idx * state_bins[1] * state_bins[2] * state_bins[3] +
             cart_vel_idx * state_bins[2] * state_bins[3] +
             pole_ang_idx * state_bins[3] +
             pole_ang_vel_idx)

    return int(index)


### 🚗 MountainCar-v0 Discretization Function

MountainCar has only **2 continuous state variables**:

 **1**. Car position

 **2**. Car velocity

The agent must learn to move between the valleys using momentum.

In [3]:
def discretize_mountaincar_state(observation, state_bins):
    """
    Convert a continuous MountainCar state into a discrete index for Q-learning.

    Parameters:
        observation : array-like of shape (2,)
            [position, velocity]
        state_bins : list or array-like of length 2
            Number of bins for each dimension, e.g. [20, 20]

    Returns:
        int
            A single integer index representing the discretized state.
    """
    car_position, car_velocity = observation

    # Define the known limits for MountainCar-v0
    position_min, position_max = -1.2, 0.6
    velocity_min, velocity_max = -0.07, 0.07

    # Create bin edges
    position_bins = np.linspace(position_min, position_max, state_bins[0] + 1)
    velocity_bins = np.linspace(velocity_min, velocity_max, state_bins[1] + 1)

    # Digitize (find which bin each variable falls into)
    pos_idx = np.digitize(car_position, position_bins) - 1
    vel_idx = np.digitize(car_velocity, velocity_bins) - 1

    # Clip indices to stay within range
    pos_idx = np.clip(pos_idx, 0, state_bins[0] - 1)
    vel_idx = np.clip(vel_idx, 0, state_bins[1] - 1)

    # Combine into one index
    index = pos_idx * state_bins[1] + vel_idx

    return int(index)


In [4]:

def initialize_q_table(state_bins, num_actions):
    """
    Create and initialize a Q-table for a discretized continuous environment.

    Parameters:
        state_bins : list or array-like
            Number of bins per state dimension, e.g. [6, 6, 12, 6]
        num_actions : int
            Number of possible discrete actions in the environment.

    Returns:
        np.ndarray
            A Q-table of shape (num_states, num_actions), initialized to zeros.
    """
    # Total number of discrete states = product of all bins
    num_states = int(np.prod(state_bins))
    
    # Initialize Q-table with zeros (or small random numbers)
    q_table = np.zeros((num_states, num_actions))

    return q_table


In [5]:


def epsilon_greedy_action(env, q_table, state_index, epsilon):
    """
    Select an action using the epsilon-greedy policy.

    This strategy balances exploration and exploitation:
    - With probability ε (epsilon), choose a random action (exploration).
    - With probability (1 - ε), choose the best-known action (exploitation).

    Parameters:
        env : gym.Env
            The environment, used to sample random actions.
        q_table : np.ndarray
            The Q-table storing estimated action values for each state.
            Shape: (num_states, num_actions)
        state_index : int
            The index of the current discrete state in the Q-table.
        epsilon : float
            The exploration rate (0 ≤ ε ≤ 1). Higher means more exploration.

    Returns:
        int
            The selected action (integer from 0 to env.action_space.n - 1).
    """

    # Generate a random number between 0 and 1
    random_value = np.random.random()

    # With probability epsilon → explore (pick a random action)
    if random_value < epsilon:
        action = env.action_space.sample()
    else:
        # Otherwise → exploit (choose the action with the highest Q-value)
        action = np.argmax(q_table[state_index])

    return action


In [6]:
def Train_SARSA(env_name, episodes_num, printout, alpha=0.1, gamma=0.99, epsilon=0.1):
    
    env = gym.make(env_name)
    
    if('CartPole' in env_name):
        state_bins = [6, 6, 12, 6]
    if('MountainCar' in env_name):
        state_bins = [20, 20]
    # Initializing the Q table
    n_actions = env.action_space.n
    Q_table = initialize_q_table(state_bins,n_actions)
    
    
    episode_rewards = []
    episode_lengths = []
    
    
    for episode in range(episodes_num):
        
        
        state, _ = env.reset()
        if('CartPole' in env_name):
            state_idx = discretize_cartpole_state(state, state_bins)
        if('MountainCar' in env_name):
            state_idx = discretize_mountaincar_state(state, state_bins)
#         state_idx = q_index(state, state_bins)
        # epsilon greedy
        action = epsilon_greedy_action(env, Q_table, state_idx, epsilon)
           

        done = False
        total_reward = 0
        steps = 0

        while not done:
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated #or truncated
            
            if('CartPole' in env_name):
                next_state_idx = discretize_cartpole_state(next_state, state_bins)
            if('MountainCar' in env_name):    
                next_state_idx = discretize_mountaincar_state(next_state, state_bins)

            # epsilon greedy
            next_action = epsilon_greedy_action(env, Q_table, next_state_idx, epsilon)
           
            # SARSA update on Q-table
            Q_table[state_idx, action] += alpha * (
                                        reward + gamma * Q_table[next_state_idx, next_action] - Q_table[state_idx, action]
                                    )

            state = next_state
            state_idx = next_state_idx
            action = next_action
            total_reward += reward
            steps += 1

        episode_rewards.append(total_reward)
        episode_lengths.append(steps)

        avg_reward = np.mean(episode_rewards[-100:])
        if(avg_reward>500 and 'CartPole' in env_name) or (avg_reward>-150 and 'MountainCar' in env_name):
            avg_length = np.mean(episode_lengths[-100:])
            print(f"Episode {episode}, Avg Reward: {avg_reward:.2f}, Avg Length: {avg_length:.2f}")
            break

        if episode % printout == 0: # printout the training progress
            avg_reward = np.mean(episode_rewards[-100:])
            avg_length = np.mean(episode_lengths[-100:])
            print(f"Episode {episode}, Avg Reward: {avg_reward:.2f}, Avg Length: {avg_length:.2f}")
            

    env.close()
    return Q_table
     

In [8]:
# Continuous states envs

env_name = 'CartPole-v1'
opt_policy = Train_SARSA(env_name, 20000, 500, alpha=0.1, gamma=0.99, epsilon=0.1)


Episode 0, Avg Reward: 10.00, Avg Length: 10.00
Episode 500, Avg Reward: 9.90, Avg Length: 9.90
Episode 1000, Avg Reward: 11.50, Avg Length: 11.50
Episode 1500, Avg Reward: 14.95, Avg Length: 14.95
Episode 2000, Avg Reward: 18.12, Avg Length: 18.12
Episode 2500, Avg Reward: 18.18, Avg Length: 18.18
Episode 3000, Avg Reward: 18.17, Avg Length: 18.17
Episode 3500, Avg Reward: 21.92, Avg Length: 21.92
Episode 4000, Avg Reward: 22.08, Avg Length: 22.08
Episode 4500, Avg Reward: 21.56, Avg Length: 21.56
Episode 5000, Avg Reward: 29.30, Avg Length: 29.30
Episode 5500, Avg Reward: 65.84, Avg Length: 65.84
Episode 5977, Avg Reward: 504.30, Avg Length: 504.30


In [9]:
# Continuous states envs

env_name = 'MountainCar-v0' 
mountaincar_bins = [20, 20]  # higher resolution helps due to sparse rewards

opt_policy = Train_SARSA(env_name, 20000, 200, alpha=0.1, gamma=0.99, epsilon=0.1)

Episode 0, Avg Reward: -8956.00, Avg Length: 8956.00
Episode 200, Avg Reward: -481.95, Avg Length: 481.95
Episode 400, Avg Reward: -293.30, Avg Length: 293.30
Episode 600, Avg Reward: -306.79, Avg Length: 306.79
Episode 800, Avg Reward: -257.52, Avg Length: 257.52
Episode 1000, Avg Reward: -202.96, Avg Length: 202.96
Episode 1200, Avg Reward: -213.62, Avg Length: 213.62
Episode 1400, Avg Reward: -222.62, Avg Length: 222.62
Episode 1600, Avg Reward: -190.78, Avg Length: 190.78
Episode 1800, Avg Reward: -204.88, Avg Length: 204.88
Episode 2000, Avg Reward: -173.40, Avg Length: 173.40
Episode 2200, Avg Reward: -213.79, Avg Length: 213.79
Episode 2400, Avg Reward: -167.66, Avg Length: 167.66
Episode 2600, Avg Reward: -171.31, Avg Length: 171.31
Episode 2800, Avg Reward: -169.70, Avg Length: 169.70
Episode 3000, Avg Reward: -183.48, Avg Length: 183.48
Episode 3200, Avg Reward: -180.56, Avg Length: 180.56
Episode 3400, Avg Reward: -173.14, Avg Length: 173.14
Episode 3600, Avg Reward: -172.95